In [1]:
! pip install transformers

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [24]:
import pandas as pd
df=pd.read_csv('/content/train.csv')
data = df[['comment_text','toxic']]
df= data[0:1000]
df.head()

,comment_text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [25]:
df.head()

,comment_text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [26]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
import torch
from transformers import TrainingArguments,Trainer
from transformers import BertTokenizer,BertForSequenceClassification
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
model=BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [28]:
model=model.to('cuda')

In [29]:
sample=['I love eating burger']
tokenizer(sample,padding=True,truncation=True,max_length=512)

{'input_ids': [[101, 1045, 2293, 5983, 15890, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1]]}

In [30]:
df.head(1)

,comment_text,toxic
0,Explanation\nWhy the edits made under my usern...,0


In [31]:
X=list(df['comment_text'])
y=list(df['toxic'])
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.2,random_state=42)
X_train_tokenized=tokenizer(X_train,padding=True,truncation=True,max_length=512)
X_val_tokenized=tokenizer(X_val,padding=True,truncation=True,max_length=512)
# padding=True -- Ensures that all tokenized inputs have same length.
# truncation=True -- If a sentence is too long, It cuts it off at the maximum allowed length.
# max_length=512 -- Sets the maximum number of tokens per input sequence to 512



In [32]:
X_train_tokenized.keys()   # BERT uses WordPiece tokenization
# input_ids -- It is numerical representation of your text
#
# attention_mask -- BERT tells which tokens to pay attention to
      # 1- Real tokens
      # 0- Padding tokens
# token_type_ids -- Segment IDs for sentence pairs

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [33]:
# Create Torch Dataset
class Dataset(torch.utils.data.Dataset):
  def __init__(self,encodings,labels=None):
    self.encodings=encodings # Tokenized Text  ['input_ids','attention_mask','token_type_ids']
    self.labels=labels       # labels (Can be None for test data) [multi-label one-hot encoded labels]

# This method fetches a single item (by index -idx)when DataLoader needs it.
# It converts tokenized data into PyTorch tensors
  def __getitem__(self,idx):
    item={key:torch.tensor(val[idx])for key, val in self.encodings.items()}
    if self.labels is not None:
      item['labels']=torch.tensor(self.labels[idx])
      return item

# Tells the how many samples are in the dataset.
  def __len__(self):
      return len(self.encodings['input_ids'])

In [34]:
train_dataset=Dataset(X_train_tokenized,y_train)
val_dataset  =Dataset(X_val_tokenized,y_val)

In [35]:
def compute_metrics(p):
  print(type(p))
  pred,labels=p
  pred=np.argmax(pred,axis=1)
  accuracy=accuracy_score(y_true=labels,y_pred=pred)
  recall=recall_score(y_true=labels,y_pred=pred)
  precision=precision_score(y_true=labels,y_pred=pred)
  f1=f1_score(y_true=labels,y_pred=pred)

  return {'accuracy':accuracy,'recall':recall,'precision':precision,'f1':f1}

In [36]:
# Define Trainer - HuggingFace API to train BERT model for multi-label classification
args=TrainingArguments(
    output_dir='output',          # Saves model checkpoints and log in the 'output/' folder
    num_train_epochs=10,
    per_device_train_batch_size=8, # Uses a batch  per GPU
    fp16=True,
    save_strategy='epoch',
    save_total_limit=2
)
trainer=Trainer(
    model=model, # BERT model for fine-tuning
    args=args,   # Training settings
    train_dataset=train_dataset,  # Training dataset
    eval_dataset=val_dataset,     #  Validation Dataset for evalueation
    compute_metrics=compute_metrics



)

In [37]:
trainer.train()

wandb: Currently logged in as: ombhandwalkar38126 (ombhandwalkar38126-student) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.088500
1000,0.000100


TrainOutput(global_step=1000, training_loss=0.044310042425990105, metrics={'train_runtime': 401.643, 'train_samples_per_second': 19.918, 'train_steps_per_second': 2.49, 'total_flos': 2104888442880000.0, 'train_loss': 0.044310042425990105, 'epoch': 10.0})

In [38]:
trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.4480799436569214,
 'eval_accuracy': 0.935,
 'eval_recall': 0.5833333333333334,
 'eval_precision': 0.8235294117647058,
 'eval_f1': 0.6829268292682927,
 'eval_runtime': 1.8177,
 'eval_samples_per_second': 110.026,
 'eval_steps_per_second': 13.753,
 'epoch': 10.0}

In [39]:
np.set_printoptions(suppress=True)

In [41]:
#text = "That was good point"
text = "go to hell"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions

SequenceClassifierOutput(loss={'logits': tensor([[-3.3906,  4.5352]], device='cuda:0', grad_fn=<ToCopyBackward0>)}, logits=tensor([[-3.3906,  4.5352]], device='cuda:0', grad_fn=<ToCopyBackward0>), hidden_states=None, attentions=None)
tensor([[3.6118e-04, 9.9964e-01]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


array([[0.00036118, 0.9996388 ]], dtype=float32)

In [42]:
trainer.save_model('CustomModel')

In [43]:
model.push_to_hub('OmBhandwalkar/YTFinetuneBert')

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/OmBhandwalkar/YTFinetuneBert/commit/a737c213348598cb0761bc085b93c48cedfc2886', commit_message='Upload BertForSequenceClassification', commit_description='', oid='a737c213348598cb0761bc085b93c48cedfc2886', pr_url=None, repo_url=RepoUrl('https://huggingface.co/OmBhandwalkar/YTFinetuneBert', endpoint='https://huggingface.co', repo_type='model', repo_id='OmBhandwalkar/YTFinetuneBert'), pr_revision=None, pr_num=None)